In [1]:
import os 
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

import pandas as pd 
import numpy as np
from src.model_utils import load_model, train_model, save_model
from src.preprocessor_utils import convert_time_to_hour
from src.eval_metric import f1_benchmark
import joblib


Trong file này ta sẽ huấn luyện mô hình với tập dữ liệu gốc và để các giá trị siêu tham số là mặc định và xem độ chính xác trong dự đoán sẽ như thế nào.

(1) Nạp dữ liệu

In [2]:
data_path = os.path.join(project_root,"data","train_goc.csv")
df = pd.read_csv(data_path)

In [3]:
print(data_path)

d:\PYTHON\data_mining\fraud-detecion-project\data\train_goc.csv


In [4]:
df.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,144549.0,2.238954,-1.724499,-2.151484,-2.577803,0.993668,3.565492,-1.785957,0.860122,-1.264003,...,-0.149574,-0.049333,0.278442,0.684735,-0.219028,-0.159167,0.037920,-0.049932,32.00,0
1,80716.0,-1.315062,1.630783,0.597001,-0.038359,-0.404580,-0.965712,0.212249,0.735381,-1.267926,...,-0.238898,-0.946773,0.323904,0.515632,-0.713000,-0.266503,-0.017794,0.051058,6.99,0
2,127116.0,1.908801,0.021184,-2.087997,0.129310,1.161468,0.605244,-0.022371,0.180296,0.283819,...,0.293609,1.095842,-0.044874,-1.689517,0.106098,0.007758,0.045164,-0.053068,14.95,0
3,91342.0,1.811257,0.316556,0.316751,3.880231,0.048454,1.020163,-0.734868,0.233651,0.681423,...,0.138869,0.700422,0.174064,0.702997,-0.212523,-0.010018,-0.017740,-0.038006,17.30,0
4,29522.0,1.358817,-1.120881,0.550266,-1.547659,-1.194950,0.275448,-1.201843,0.212889,-2.094285,...,-0.340972,-0.636442,0.252758,-0.344160,-0.064282,-0.439622,0.062524,0.013095,24.00,0


Sau khi nạp dữ liệu đầu vào thì t nhận thấy là data ở cột time và cột amount chưa đc preprocessing nên ta cần nạp thêm pipeline của preprocessor để tiền xử lý dữ liệu.

trước đó ta sẽ chia làm 2 tập x và y trước
+ trong đó x chứa các features của mô hình. 
+ và y là kết quả.

In [5]:
x = df.drop('Class', axis = 1)
y = df['Class']

In [6]:
preprocessor_path = os.path.join(project_root,"data","creditcard_preprocessor.pkl")
preprocessor = joblib.load(preprocessor_path)

In [7]:
# 1. Biến đổi dữ liệu ra mảng numpy
data_array = preprocessor.transform(x)

# 2. Định nghĩa lại tên cột theo đúng thứ tự trong Pipeline của bạn
# (Thứ tự: Time -> Amount -> V1...V28)
cols = ['Time', 'Amount'] + [f'V{i}' for i in range(1, 29)]

# 3. Tạo DataFrame mới từ mảng đó
df_pre = pd.DataFrame(data_array, columns=cols)

# 4. Bây giờ bạn mới dùng được .head()
df_pre.head(5)

,Time,Amount,V1,V2,V3,V4,V5,V6,V7,V8,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
0,0.333791,-0.229434,1.147198,-1.042032,-1.428821,-1.817824,0.727198,2.695677,-1.489310,0.735746,...,-0.391838,-0.427196,-0.208573,-0.068205,0.445379,1.130202,-0.420363,-0.331281,0.092782,-0.156193
1,1.363148,-0.331197,-0.677911,0.991948,0.394668,-0.025642,-0.301198,-0.728240,0.175615,0.629070,...,0.481434,-0.089833,-0.332248,-1.307141,0.518036,0.851069,-1.367338,-0.554255,-0.049864,0.153831
2,-0.524006,-0.298809,0.977654,0.016205,-1.386701,0.092688,0.850613,0.458823,-0.019872,0.154370,...,-1.223343,-0.277973,0.405040,1.512732,-0.071350,-2.788905,0.202924,0.015479,0.111329,-0.165820
3,-2.239601,-0.289247,0.927561,0.195261,0.208736,2.739855,0.032003,0.772348,-0.613532,0.199999,...,-2.082239,-0.301091,0.190793,0.966846,0.278559,1.160348,-0.407893,-0.021447,-0.049724,-0.119582
4,-1.038685,-0.261985,0.695218,-0.676118,0.363661,-1.090812,-0.882507,0.209618,-1.002620,0.182243,...,-0.728156,-0.477066,-0.473576,-0.878722,0.404329,-0.568164,-0.123705,-0.913882,0.155776,0.037290


Thử với mô hình logistic regression

In [8]:
model = train_model(df_pre,y,'lr')


Cảnh báo không tình thấy file: configs/lr_config.json. sử dụng tham số mặc định
Training LR...
Train successful!!


In [ ]:
score = f1_benchmark(model, df_pre, y)

In [10]:
print(f"Độ chính xác: {score['accuracy']}")
print(f"Độ chính xác f1: {score['f1']}")

Độ chính xác: 0.9992025729139131
Độ chính xác f1: 0.7167449139280125
